In [16]:
import pandas as pd
from wordfreq import word_frequency ,top_n_list,get_frequency_dict

In [17]:
from symspellpy import SymSpell, Verbosity
from itertools import islice

In [19]:
from pandarallel import pandarallel

pandarallel.initialize(progress_bar=True,nb_workers=4)

INFO: Pandarallel will run on 4 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [20]:
df = pd.read_csv('final.csv')

In [21]:
df

,path,id,source,audio,sentence,duration,snr,wav2vec2,2gram,3gram,4gram,2gram_label,2gram_wer,wav2vec2_label,wav2vec2_wer,3gram_label,3gram_wer,4gram_label,4gram_wer
0,/kaggle/input/denoiser-for-benchmark/raw/train...,30991599,train,noisy,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,5.544,-44.277168,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'D', 'S', 'C', 'C', 'C', ...",15,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0
1,/kaggle/input/denoiser-for-benchmark/raw/train...,30991687,train,noisy,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,4.032,-46.880779,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0
2,/kaggle/input/denoiser-for-benchmark/raw/train...,30991851,train,noisy,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,4.320,-56.581566,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,"['S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', ...",36,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33
3,/kaggle/input/denoiser-for-benchmark/raw/train...,30992090,train,noisy,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,4.284,-29.025093,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12
4,/kaggle/input/denoiser-for-benchmark/raw/train...,30992661,train,noisy,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,5.760,-32.322632,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,"['S', 'C', 'C', 'C', 'C', 'I', 'S', 'C', 'C']",33,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49147,/kaggle/input/denoiser-for-benchmark/denoised/...,056711c865,openslr,denoised,রাজনৈতিক সমাবেশ,3.168,-37.201962,রাজনৈতিক সমাপেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,"['C', 'C']",0,"['C', 'S']",50,"['C', 'C']",0,"['C', 'C']",0
49148,/kaggle/input/denoiser-for-benchmark/denoised/...,a3230c3684,openslr,denoised,এমন সংসদ সদস্য,3.348,-36.750912,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0
49149,/kaggle/input/denoiser-for-benchmark/denoised/...,7f87c122cb,openslr,denoised,ব্লেড গেল,2.736,-26.636936,ল্লেড গেলো,ব্লেড গেল,ব্লেড গেল,ব্লেড গেল,"['C', 'C']",0,"['S', 'S']",100,"['C', 'C']",0,"['C', 'C']",0
49150,/kaggle/input/denoiser-for-benchmark/denoised/...,a6463ecbb0,openslr,denoised,তাদের ছেলেমেয়েদের,4.536,-18.375225,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,"['C', 'I', 'S']",66,"['C', 'I', 'S']",66,"['C', 'I', 'S']",66,"['C', 'I', 'S']",66


In [22]:
sym_spell_word_segmentation = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = 'asr_bangla/csv_files/prothom_alo_word_freq.txt'
sym_spell_word_segmentation.load_dictionary(dictionary_path, 0, 1,separator=" ")

True

In [23]:
sym_spell = SymSpell(max_dictionary_edit_distance=5, prefix_length=7)
dictionary_path = 'asr_bangla/csv_files/prothom_alo_word_freq.txt'
sym_spell.load_dictionary(dictionary_path, 0, 1,separator=" ")

True

In [24]:
def word_segmentation(input_term):
    result = sym_spell_word_segmentation.word_segmentation(input_term)
    return result.corrected_string

In [25]:
def dictionary_(word):
    suggestions = sym_spell.lookup(
    word, Verbosity.CLOSEST,max_edit_distance=2, include_unknown=True)
    for suggestion in suggestions:
        return str(suggestion).split(',')[0]

In [31]:
def lookup(sen):
    words = sen.split()
    m=[]
    for wow in words:
        if len(wow)>16:
            s=word_segmentation(wow)
            j = s.split()
            for n in j:
                m.append(n)
        else:
            m.append(wow)
    l=[]
    for wow in m:
        if (word_frequency(wow,'bn',wordlist='large',minimum=0.0) == 0.0):
            s=dictionary_(wow)
            l.append(s)
        else:
            l.append(wow)
    return ' '.join(l)

In [32]:
df

,path,id,source,audio,sentence,duration,snr,wav2vec2,2gram,3gram,4gram,2gram_label,2gram_wer,wav2vec2_label,wav2vec2_wer,3gram_label,3gram_wer,4gram_label,4gram_wer
0,/kaggle/input/denoiser-for-benchmark/raw/train...,30991599,train,noisy,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,5.544,-44.277168,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'D', 'S', 'C', 'C', 'C', ...",15,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0
1,/kaggle/input/denoiser-for-benchmark/raw/train...,30991687,train,noisy,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,4.032,-46.880779,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0
2,/kaggle/input/denoiser-for-benchmark/raw/train...,30991851,train,noisy,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,4.320,-56.581566,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,"['S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', ...",36,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33
3,/kaggle/input/denoiser-for-benchmark/raw/train...,30992090,train,noisy,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,4.284,-29.025093,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12
4,/kaggle/input/denoiser-for-benchmark/raw/train...,30992661,train,noisy,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,5.760,-32.322632,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,"['S', 'C', 'C', 'C', 'C', 'I', 'S', 'C', 'C']",33,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49147,/kaggle/input/denoiser-for-benchmark/denoised/...,056711c865,openslr,denoised,রাজনৈতিক সমাবেশ,3.168,-37.201962,রাজনৈতিক সমাপেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,"['C', 'C']",0,"['C', 'S']",50,"['C', 'C']",0,"['C', 'C']",0
49148,/kaggle/input/denoiser-for-benchmark/denoised/...,a3230c3684,openslr,denoised,এমন সংসদ সদস্য,3.348,-36.750912,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0
49149,/kaggle/input/denoiser-for-benchmark/denoised/...,7f87c122cb,openslr,denoised,ব্লেড গেল,2.736,-26.636936,ল্লেড গেলো,ব্লেড গেল,ব্লেড গেল,ব্লেড গেল,"['C', 'C']",0,"['S', 'S']",100,"['C', 'C']",0,"['C', 'C']",0
49150,/kaggle/input/denoiser-for-benchmark/denoised/...,a6463ecbb0,openslr,denoised,তাদের ছেলেমেয়েদের,4.536,-18.375225,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,তাদের ছেলে মেয়েদের,"['C', 'I', 'S']",66,"['C', 'I', 'S']",66,"['C', 'I', 'S']",66,"['C', 'I', 'S']",66


In [33]:
print(list(islice(sym_spell.words.items(), 5)))

[('পশ্চিমবঙ্গের', 2525), ('বরখাস্ত', 2021), ('হওয়া', 28354), ('মন্ত্রী', 6635), ('পার্থ', 1351)]


In [34]:
df=df.dropna()

In [35]:
df.columns

Index(['path', 'id', 'source', 'audio', 'sentence', 'duration', 'snr',
       'wav2vec2', '2gram', '3gram', '4gram', '2gram_label', '2gram_wer',
       'wav2vec2_label', 'wav2vec2_wer', '3gram_label', '3gram_wer',
       '4gram_label', '4gram_wer'],
      dtype='object')

In [36]:
df['wav2vec2_sym'] = df['wav2vec2'].parallel_apply(lookup)
df['2gram_sym'] = df['2gram'].parallel_apply(lookup)
df['3gram_sym'] = df['3gram'].parallel_apply(lookup)
df['4gram_sym'] = df['4gram'].parallel_apply(lookup)

In [37]:
df

,path,id,source,audio,sentence,duration,snr,wav2vec2,2gram,3gram,...,wav2vec2_label,wav2vec2_wer,3gram_label,3gram_wer,4gram_label,4gram_wer,wav2vec2_sym,2gram_sym,3gram_sym,4gram_sym
0,/kaggle/input/denoiser-for-benchmark/raw/train...,30991599,train,noisy,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,5.544,-44.277168,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,...,"['C', 'C', 'C', 'C', 'D', 'S', 'C', 'C', 'C', ...",15,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', 'C', ...",0,ছানার সাথে চিনি বা গুর্মীশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...,ছানার সাথে চিনি বা গুড় মিশিয়ে ছাঁচে ফেলে সন্দে...
1,/kaggle/input/denoiser-for-benchmark/raw/train...,30991687,train,noisy,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,4.032,-46.880779,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'C', 'C', 'C']",0,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক,তিনি ট্রেড ইউনিয়ন আন্দোলনের অন্যতম প্রধান নায়ক
2,/kaggle/input/denoiser-for-benchmark/raw/train...,30991851,train,noisy,গরীব মানুষকে সে মানুষ বলেই মনে করে না এবং তুচ্...,4.320,-56.581566,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,...,"['S', 'S', 'S', 'C', 'S', 'C', 'C', 'C', 'C', ...",36,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,"['I', 'S', 'S', 'C', 'C', 'S', 'C', 'C', 'C', ...",33,গড়ি মানুষ কেসে মানুষ বলি মনে করে না এবং তুচ্ছত...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...,গরিব মানুষ কি সে মানুষ বলে মনে করে না এবং তুচ্...
3,/kaggle/input/denoiser-for-benchmark/raw/train...,30992090,train,noisy,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,4.284,-29.025093,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,...,"['C', 'C', 'C', 'C', 'C', 'C', 'C', 'C']",0,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,"['C', 'C', 'C', 'C', 'D', 'C', 'C', 'C']",12,ব্যাটের বেশ দূর থেকে এ সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি,ব্যাটের বেশ দূর থেকে সুইং করতেন তিনি
4,/kaggle/input/denoiser-for-benchmark/raw/train...,30992661,train,noisy,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,5.760,-32.322632,অধ্যাবধী এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,...,"['S', 'C', 'C', 'C', 'C', 'I', 'S', 'C', 'C']",33,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,"['C', 'C', 'C', 'I', 'S', 'C', 'I', 'S', 'C', ...",40,অদ্যাবধি এ সংগ্রহটি প্রথমশ্রেণীর ক্রিকেটে বিশ্...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...,অদ্যাবধি এ সংগ্রহটি প্রথম শ্রেণির ক্রিকেটে বিশ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49147,/kaggle/input/denoiser-for-benchmark/denoised/...,056711c865,openslr,denoised,রাজনৈতিক সমাবেশ,3.168,-37.201962,রাজনৈতিক সমাপেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,...,"['C', 'S']",50,"['C', 'C']",0,"['C', 'C']",0,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ,রাজনৈতিক সমাবেশ
49148,/kaggle/input/denoiser-for-benchmark/denoised/...,a3230c3684,openslr,denoised,এমন সংসদ সদস্য,3.348,-36.750912,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,...,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,"['C', 'C', 'C']",0,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য,এমন সংসদ সদস্য
49149,/kaggle/input/denoiser-for-benchmark/denoised/...,7f87c122cb,ope

In [47]:
df[(df['4gram_wer'] > 80) & (df['4gram_wer']<90)  ][['sentence','4gram','4gram_wer','4gram_sym']]

,sentence,4gram,4gram_wer,4gram_sym
966,এর একটি জাহাজকে সংস্কার করে প্রদর্শনীতে রাখা হ...,এর এক জামদানিতে,87,এর এক জামদানিতে
1354,পরবর্তীতে ইংল্যান্ডের প্রথমশ্রেণীর ক্রিকেটে অং...,পরবর্তীতে ইংল্যান্ড প্রথম শ্রেণির টেস্ট খেতে অনশন,85,পরবর্তীতে ইংল্যান্ড প্রথম শ্রেণির টেস্ট খেতে অনশন
1767,কর্মধারয় সমাস কয়েক প্রকার হতে পারে,কর্ম ধরে সমাজ কয়েক প্রবাহিত হয়,85,কর্ম ধরে সমাজ কয়েক প্রবাহিত হয়
2366,অপ্রাপ্তবয়স্ক ময়ূর স্ত্রী ময়ূরের মত,অপ্রাপ্ত বয়স্ক মুর স্ত্রী মহরের মতোন,83,অপ্রাপ্ত বয়স্ক মুর স্ত্রী মহরের মতোন
2487,সৃষ্টি প্রার্থনা প্রতিফলন এবং মানবজাতির সেবায় ...,সৃষ্টির পাচন্দর এবং মান্য সালে দিকে তার স্রষ্ট...,83,সৃষ্টির পাচন্দর এবং মান্য সালে দিকে তার স্রষ্ট...
...,...,...,...,...
46362,ডিএইচফ্রিস্টন যিনি হোমসকে,ডি এই সিস্ট ন যিনি হোমস কে,85,ডি এই সিস্ট ন যিনি হোমস কে
46402,দুই জন ব্রাহ্মণ কৃপ ও অশ্বত্থামা,দুইজন ভ্রামন ফিলপ ও অশ্বথাম,83,দুইজন ভ্রামন ফিলপ ও অশ্বথ
47183,অনেকক্ষণ তাঁর জ্ঞান হয় নি,অনেক কোন তার জ্ঞান হয়নি,83,অনেক কোন তার জ্ঞান হয়নি
47465,যথাক্রমে কোটি লাখ,যত কমে দুই কোটি ছত্রিশ লা,83,যত কমে দুই কোটি ছত্রিশ লা


In [38]:
df.to_csv('symspell_nov.csv',index=False)